In [30]:
import geopandas as gpd
import boto3

In [31]:
bucket_name = 'opera-calval-database-dswx'
bucket_name_staging = 'opera-calval-database-dswx-staging'

In [32]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

In [33]:
bucket = s3.Bucket(bucket_name_staging)

## List objects within a given prefix
for obj in bucket.objects.filter(Delimiter='/', Prefix='pending/'):
    print(obj.key)

pending/20220719_211840_imagecalc.geojson


In [34]:
pending_key = 'pending/20220719_211840_imagecalc.geojson'

In [35]:
obj = s3.Object(bucket_name_staging,pending_key)
pending_data = obj.get()['Body']
pending_gdf = gpd.read_file(pending_data)
pending_gdf.head()

,bucket,calc_type,fitness,image_calc_name,image_name,oversight_level,previous_name,processing_level,public,s3_key_imagecalc,s3_key_logfile,s3_key_training,upload_date,geometry
0,opera-calval-database-dswx-staging,Supervised Classification,None,20210912_034049_22_2421_supMD,20210912_034049_22_2421,None,None,Intermediate,True,pending/uploads/20210912_034049_22_2421_supMD2...,pending/uploads/20210912_034049_22_2421_supMD0...,pending/uploads/20210912_034049_22_2421_supMDt...,20220719_211840,"POLYGON ((97.44674 49.52974, 97.37427 49.34386..."


In [36]:
imagecalc_gdf = gpd.read_file(s3.Object(bucket_name,'image_calc.geojson').get()['Body'])
imagecalc_gdf.head()

image_gdf = gpd.read_file(s3.Object(bucket_name,'image.geojson').get()['Body'])
image_gdf.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,s3_bucket,s3_key_image,site_coverage,site_name,timeDelta_days,geometry
0,0.0,HLS.S30.T10VEK.2021268T192141.v2.0,2021-09-25T19:17:08,20210925_191708_94_2408,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/3_42/20210925_191708_94_2408/20210925_191...,84.975663,3_42,0.007904,"POLYGON ((-122.94132 57.97664, -123.03517 57.7..."
1,0.0,HLS.L30.T18UXG.2021300T154817.v2.0,2021-10-27T15:14:45,20211027_151445_21_2231,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_21/20211027_151445_21_2231/20211027_151...,52.216780,4_21,0.023288,"POLYGON ((-71.93982 55.28796, -72.02213 55.104..."
2,0.0,HLS.S30.T18FYH.2021302T141741.v2.0,2021-10-30T14:26:13,20211030_142613_41_227b,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/2_8/20211030_142613_41_227b/20211030_1426...,74.756870,2_8,0.992265,"POLYGON ((-71.22036 -51.94097, -71.30174 -52.1..."
3,0.0,HLS.S30.T36LWH.2021268T073641.v2.0,2021-09-26T07:24:14,20210926_072414_71_2233,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_3/20210926_072414_71_2233/20210926_0724...,97.600689,4_3,0.972010,"POLYGON ((33.42779 -16.15910, 33.38993 -16.342..."
4,0.0,HLS.S30.T15VUK.2021244T175911.v2.0,2021-09-01T17:52:05,20210901_175205_71_2426,PSB.SD,planetscope,3.0,opera-calval-database-dswx-private,data/4_34/20210901_175205_71_2426/20210901_175...,96.416886,4_34,0.009994,"POLYGON ((-96.94381 62.88876, -97.07609 62.707..."


In [37]:
source_image_name = pending_gdf.image_name.iloc[0]
filename = pending_gdf.image_calc_name.iloc[0]
source_bucket = pending_gdf.bucket.iloc[0]
src_imagecalc = pending_gdf.s3_key_imagecalc.iloc[0]
src_logfile = pending_gdf.s3_key_logfile.iloc[0]
src_training = pending_gdf.s3_key_training.iloc[0]

In [38]:
site = image_gdf[image_gdf.image_name == source_image_name].site_name.iloc[0]
site

'4_8'

In [39]:
s3_folder_path = 'data/site/'+site+'/image/'+source_image_name+'/image_calc/'+filename+'/'
s3_key_image = s3_folder_path+filename+'.tif'
response = s3.meta.client.copy({'Bucket':source_bucket,'Key':src_imagecalc}, bucket_name, s3_key_image)
if src_logfile != None:
    s3_key_logfile = s3_folder_path+filename+'_logfile.docx'
    response = s3.meta.client.copy({'Bucket':source_bucket,'Key':src_logfile}, bucket_name, s3_key_logfile)
else:
    s3_key_logfile = None
if src_training != None:
    s3_key_training = s3_folder_path+filename+'_training.scp'
    response = s3.meta.client.copy({'Bucket':source_bucket,'Key':src_training}, bucket_name, s3_key_training)
else:
    s3_key_training = None
    
pending_gdf['s3_key_imagecalc'] = s3_key_image
pending_gdf['s3_key_logfile'] = s3_key_logfile
pending_gdf['s3_key_training'] = s3_key_training

pending_gdf['bucket'] = bucket_name

imagecalc_upd = imagecalc_gdf.append(pending_gdf,ignore_index=True)

imagecalc_upd_bytes = bytes(imagecalc_upd.to_json(drop_id=True).encode('UTF-8'))
s3object = s3.Object(bucket_name,'image_calc.geojson')
s3object.put(Body=imagecalc_upd_bytes)



/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


{'ResponseMetadata': {'RequestId': 'MD83MWPGYG1N6JM3',
  'HostId': 'HRndBOZwojSokR5rYpJG5h3JpYBTDBo2wBe60Cz5aHxvi13PHlDk+fcK2J9RJs0pgOK9Ltye7sk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'HRndBOZwojSokR5rYpJG5h3JpYBTDBo2wBe60Cz5aHxvi13PHlDk+fcK2J9RJs0pgOK9Ltye7sk=',
   'x-amz-request-id': 'MD83MWPGYG1N6JM3',
   'date': 'Wed, 20 Jul 2022 19:32:15 GMT',
   'etag': '"1fe06b96c7c8fc89a89d9c165decaa26"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"1fe06b96c7c8fc89a89d9c165decaa26"'}